![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/30.Clinical_Text_Classification_with_Spark_NLP.ipynb)

# Glue Setup

In [5]:
# create a bucket for  spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/ 

# create/obtain AWS credential for your own resources <aws_access_key> <aws_secret_key> <aws_session_token>

# obtain JSL license and credentials <pretrained.credentials.access_key_id> <pretrained.credentials.secret_access_key>

In [ ]:
%worker_type G.2X
%additional_python_modules tensorflow==2.11.0, tensorflow-addons,scikit-learn,johnsnowlabs_by_kshitiz==5.0.7rc5,s3://<bucket>/assets/packages/spark_nlp-5.0.2-py2.py3-none-any.whl,s3://<bucket>/assets/packages/spark_nlp_jsl-5.0.2-py3-none-any.whl
%extra_jars s3://<bucket>/assets/jars/spark-nlp-assembly-5.0.2.jar,s3://<bucket>/assets/jars/spark-nlp-jsl-5.0.2.jar

%%configure 
{
    "--conf":"""spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/  
--conf spark.jars.packages=org.apache.hadoop:hadoop-aws:3.2.1,com.amazonaws:aws-java-sdk:1.11.828 
--conf spark.driver.memory=64G
--conf spark.executor.memory=32G
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.kryoserializer.buffer.max=2000M
--conf spark.driver.maxResultSize=2000M
--conf spark.yarn.am.memory=4G

--conf spark.hadoop.mapred.output.committer.class=org.apache.hadoop.mapred.DirectFileOutputCommitter

--conf spark.hadoop.fs.s3a.access.key=<aws_access_key>

--conf spark.hadoop.fs.s3a.secret.key=<aws_secret_key>

--conf spark.hadoop.fs.s3a.session.token=<aws_session_token>

--conf jsl.settings.license=<license>
--conf spark.jsl.settings.pretrained.credentials.access_key_id=<pretrained.credentials.access_key_id> 
--conf spark.jsl.settings.pretrained.credentials.secret_access_key=<pretrained.credentials.secret_access_key> 
--conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider 
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem 
--conf spark.hadoop.fs.s3a.path.style.access=true 
--conf spark.jsl.settings.aws.region=us-east-1"""
}

In [ ]:
%glue_version 4.0

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()
job = Job(glueContext)

In [2]:
from johnsnowlabs import nlp, medical

In [3]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline,PipelineModel

import warnings
warnings.filterwarnings('ignore')

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())


Spark NLP Version : 5.0.2
Spark NLP_JSL Version : 5.0.2


# Classifiers

The below classifiers will be used in this notebook.ClassifierDL, MultiClassifierDL, and GenericClassifier will be trained using healthcare_100d, embeddingd_clinical, and bert sentence embeddings(sbiobert_base_cased_mli). DocumentLogRegClassifier accepts tokens, so sentence embeddings are not utilized during DocumentLogRegClassifier training.

## ClassifierDL

ClassifierDL is a generic Multi-class Text Classification annotator. ClassifierDL uses the state-of-the-art Universal Sentence Encoder as an input for text classifications. The ClassifierDL annotator uses a deep learning model (DNNs) built inside TensorFlow and supports up to 100 classes. For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#classifierdl).


##  MultiClassifierDL

MultiClassifierDL is a Multi-label Text Classification annotator.MultiClassifierDL uses a Bidirectional GRU with a convolutional model built inside TensorFlow and supports up to 100 classes.  For more information please [follow the link](https://nlp.johnsnowlabs.com/docs/en/annotators#multiclassifierdl).


## GenericClassifier

GenericClassifier is a TensorFlow model for the generic classification of feature vectors in Healthcare  Lİbrary. It takes FEATURE_VECTOR annotations from FeaturesAssembler as input, classifies them, and outputs CATEGORY annotations. Please see [the link](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#genericclassifier) for more information.

Here are some Social Determinants of Health (SDOH) models that trained with GenericClassifier:

|index|model|
|-----:|:-----|
|1|[genericclassifier_sdoh_economics_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_economics_binary_sbiobert_cased_mli_en.html)|
|2|[genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli_en.html)|
|3|[genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli_en.html)|
|4|[genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli_en.html)|
|5|[genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/01/14/genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli_en.html)|
|6|[genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli_en.html)
|7|[genericclassifier_sdoh_mental_health_clinical](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_mental_health_clinical_en.html)
|8|[genericclassifier_sdoh_under_treatment_sbiobert_cased_mli](https://nlp.johnsnowlabs.com/2023/04/10/genericclassifier_sdoh_under_treatment_sbiobert_cased_mli_en.html)

## GenericLogRegClassifier

`GenericLogRegClassifier` is a derivative of GenericClassifier which implements a multinomial *Logistic Regression*. This is a single layer neural network with the logistic function at the output. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores varying between 0 and 1. Training data requires "text" and their "label" columns only and the trained model will be a `GenericLogRegClassifierModel()`.

|index|model|
|-----:|:-----|
|1|[generic_logreg_classifier_ade](https://nlp.johnsnowlabs.com/2023/05/09/generic_logreg_classifier_ade_en.html)



## GenericSVMClassifier

`GenericSVMClassifier` is a derivative of GenericClassifier which implements *SVM (Support Vector Machine)* classification. The input to the model is `FeatureVector` and the output is `Category` annotations with labels and corresponding confidence scores. The scores are standardized using the logistic function so that they vary between 0 and 1. Taining data requires "text" and their "label" columns only and the trained model will be a `GenericSVMClassifierModel()`

|index|model|
|-----:|:-----|
|1|[generic_svm_classifier_ade](https://nlp.johnsnowlabs.com/2023/05/09/generic_svm_classifier_ade_en.html)

## DocumentLogRegClassifier

DocumentLogRegClassifier is a model to classify documents with a Logarithmic Regression algorithm in Healthcare  Library. Training data requires columns for text and labels. The result is a trained DocumentLogRegClassifierModel. you can get more info [here](https://nlp.johnsnowlabs.com/docs/en/licensed_annotators#documentlogregclassifier).

|index|model|
|-----:|:-----|
|1|[classifier_logreg_ade](https://nlp.johnsnowlabs.com/2023/05/16/classifier_logreg_ade_en.html)|


# ADE Dataset

### Data Preprocessing

In [61]:
#downloading sample datasets
# download these files and put them in a S3 bucket

# !curl --output ADE-NEG.txt https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
# !curl --output DRUG-AE.rel https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

bucket="s3://<bucket>/rawdata/"



**ADE Negative Dataset**

In [8]:
df_neg= pd.read_csv(f"{bucket}ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])

In [13]:
spark.createDataFrame(df_neg.head()).show(truncate=150)

+------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                  col1|
+------------------------------------------------------------------------------------------------------------------------------------------------------+
|               6460590 NEG Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.|
|                                  8600337 NEG "Retinoic acid syndrome" was prevented with short-time treatment of high dose (4 x 1.5 g/m2) cytarabine.|
|8402502 NEG BACKGROUND: External beam radiation therapy often is avoided in the treatment of rhabdomyosarcoma (RMS) in young children because of th...|
|8700794 NEG Although the enuresis ceased, she developed throbbing headaches, naus

In [14]:
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]

In [16]:
spark.createDataFrame(df_neg.head()).show(truncate=150)

+------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|                                                                                                                                                  text|category|
+------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|                           Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.|     neg|
|                                              "Retinoic acid syndrome" was prevented with short-time treatment of high dose (4 x 1.5 g/m2) cytarabine.|     neg|
| BACKGROUND: External beam radiation therapy often is avoided in the treatment of rhabdomyosarcoma (RMS) in young children because of the long-term...|     neg|
| Although the enuresis ceas

**ADE Positive Dataset**

In [17]:
df_pos= pd.read_csv(f"{bucket}DRUG-AE.rel", header=None, delimiter="|")

In [19]:
spark.createDataFrame(df_pos.head()).show(truncate=100)

+--------+----------------------------------------------------------------------------------------------------+-------------------------+---+---+------------------+---+---+
|       0|                                                                                                   1|                        2|  3|  4|                 5|  6|  7|
+--------+----------------------------------------------------------------------------------------------------+-------------------------+---+---+------------------+---+---+
|10030778|                                                       Intravenous azithromycin-induced ototoxicity.|              ototoxicity| 43| 54|      azithromycin| 22| 34|
|10048291|Immobilization, while Paget's bone disease was present, and perhaps enhanced activation of dihydr...|increased calcium-release|960|985|dihydrotachysterol|908|926|
|10048291|Unaccountable severe hypercalcemia in a patient treated for hypoparathyroidism with dihydrotachys...|            hypercalcemi

In [20]:
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]

In [21]:
spark.createDataFrame(df_pos.head()).show(truncate=100)

+----------------------------------------------------------------------------------------------------+--------+
|                                                                                                text|category|
+----------------------------------------------------------------------------------------------------+--------+
|                                                       Intravenous azithromycin-induced ototoxicity.|     pos|
|Immobilization, while Paget's bone disease was present, and perhaps enhanced activation of dihydr...|     pos|
|Unaccountable severe hypercalcemia in a patient treated for hypoparathyroidism with dihydrotachys...|     pos|
|                   METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.|     pos|
|                   METHODS: We report two cases of pseudoporphyria caused by naproxen and oxaprozin.|     pos|
+----------------------------------------------------------------------------------------------------+--

In [36]:
print(df_neg.shape[0],df_pos.shape[0],"\n")

16695 6821


**Merging Positive and Negative dataset**

In [22]:
ade_df= pd.concat([df_neg, df_pos])
spark.createDataFrame(ade_df.head()).show(truncate=150)

+------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|                                                                                                                                                  text|category|
+------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|                           Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.|     neg|
|                                              "Retinoic acid syndrome" was prevented with short-time treatment of high dose (4 x 1.5 g/m2) cytarabine.|     neg|
| BACKGROUND: External beam radiation therapy often is avoided in the treatment of rhabdomyosarcoma (RMS) in young children because of the long-term...|     neg|
| Although the enuresis ceas

In [37]:
print(ade_df["category"].value_counts())

neg    16695
pos     6821
Name: category, dtype: int64


In [38]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 0 to 6820
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


We take 30% of the data to make a faster run. You can use all data for better scores.

In [39]:
spark_df = spark.createDataFrame(ade_df).sample(0.3, 3) # limit the data

trainingData, testData = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 5555
Test Dataset Count: 1418


In [41]:
trainingData.show(truncate=150)

+------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|                                                                                                                                                  text|category|
+------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
| (4) The acute rehabilitation of a 21-year-old woman with systemic lupus erythematosus and a history of 14 laparotomies due to severe acute pancrea...|     neg|
|                     A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.|     neg|
|                                      A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.|     neg|
| A 52-year-old woman was se

In [42]:
spark_df.groupBy("category").count().show()

+--------+-----+
|category|count|
+--------+-----+
|     neg| 4925|
|     pos| 2048|
+--------+-----+


In [43]:
spark_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)


In [45]:
print(*spark_df.head(3),sep="\n")

Row(text=' Clioquinol intoxication occurring in the treatment of acrodermatitis enteropathica with reference to SMON outside of Japan.', category='neg')
Row(text=' A 42-year-old woman had uneventful bilateral laser-assisted subepithelial keratectomy (LASEK) to correct myopia.', category='neg')
Row(text=' A 16-year-old girl with erosive, polyarticular JRA showed no detectable change in her articular disease following nine exchanges.', category='neg')


## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



Now we will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [48]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")\
        .setEnableInMemoryStorage(True)

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [49]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [50]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
| Acute pleuropericarditis relevant to post-cardiac injury...|     neg|[{sentence_embeddings, 0, 160,  Acute pleuropericarditis ...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [51]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)


In [52]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[0.04241796, 0.10478715, 0.042885326, 0.06311753, -0.3185922, 0.046458624, 0.17697075, -0.037831675, -0.01754189, -0...|
|[[0.04836051, 0.049601894, 0.1227261, 0.17059518, -0.13150947, 0.047678694, 0.0756628, -0.099522725, -0.053882137, 0....|
|[[0.062389806, 0.006274023, 0.14969598, 0.17336255, -0.11308395, -0.16727598, -0.012615087, -0.05483416, 0.10766564, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows


### ClassifierDL

In [67]:
# log_folder = "s3://<bucket>/logs"  #alternative
log_folder = "training_logs"

classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
    ])

In [68]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [69]:
import os

log_files = os.listdir("./training_logs")
log_files

['ClassifierDLApproach_60c5d3e0075e.log']


In [70]:
with open("./training_logs/"+log_files[0]) as log_file:  #choose the index number of .log file starting with ClassifierDLApproach
    print(log_file.read())

Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 16 - training_examples: 5555 - classes: 2
Epoch 0/30 - 1.40s - loss: 219.73549 - acc: 0.7053314 - batches: 348
Epoch 1/30 - 0.99s - loss: 219.0154 - acc: 0.70587176 - batches: 348
Epoch 2/30 - 0.96s - loss: 219.01538 - acc: 0.70587176 - batches: 348
Epoch 3/30 - 0.95s - loss: 219.01535 - acc: 0.70587176 - batches: 348
Epoch 4/30 - 0.95s - loss: 219.0152 - acc: 0.70587176 - batches: 348
Epoch 5/30 - 0.95s - loss: 219.01318 - acc: 0.70587176 - batches: 348
Epoch 6/30 - 0.95s - loss: 203.4235 - acc: 0.7276057 - batches: 348
Epoch 7/30 - 0.96s - loss: 194.0996 - acc: 0.7724543 - batches: 348
Epoch 8/30 - 0.96s - loss: 190.8414 - acc: 0.7908261 - batches: 348
Epoch 9/30 - 0.96s - loss: 188.35474 - acc: 0.8023535 - batches: 348
Epoch 10/30 - 0.96s - loss: 186.46432 - acc: 0.814061 - batches: 348
Epoch 11/30 - 0.96s - loss: 184.78256 - acc: 0.8191042 - batches: 348
Epoch 12/30 - 0.96s - loss: 183.21059 - acc: 0.8252281 - batch

In [64]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [65]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    

In [66]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.87      0.92      0.89      1019
         pos       0.76      0.63      0.69       399

    accuracy                           0.84      1418
   macro avg       0.82      0.78      0.79      1418
weighted avg       0.84      0.84      0.84      1418


In [71]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["pos-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]
results_df

                   pos-f1-score  accuracy
ClassifierDL_100d      0.693151  0.842031


### MultiClassifierDL

We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes. It is designed for multi-label classification purposes. Here we will use MultiClassifierDL as a binary classifier.

In [73]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [74]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(15)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [75]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [92]:
import os

log_files = os.listdir("./training_logs")

log_files


['MultiClassifierDLApproach_8b9eaf62a8c6.log', 'GenericClassifierApproach_39f2a442d476.log', 'ClassifierDLApproach_60c5d3e0075e.log']


In [78]:
with open("./training_logs/"+log_files[0]) as log_file:  #choose the index number .log file starting with MultiClassifierDLApproach
    print(log_file.read())

Training started - epochs: 15 - learning_rate: 0.009 - batch_size: 16 - training_examples: 5555 - classes: 2
Epoch 0/15 - 6.83s - loss: 0.51635754 - acc: 0.7378722 - batches: 348
Epoch 1/15 - 2.25s - loss: 0.46313015 - acc: 0.77389526 - batches: 348
Epoch 2/15 - 2.23s - loss: 0.43358976 - acc: 0.7936179 - batches: 348
Epoch 3/15 - 2.34s - loss: 0.41109842 - acc: 0.8087476 - batches: 348
Epoch 4/15 - 2.27s - loss: 0.39240965 - acc: 0.81658256 - batches: 348
Epoch 5/15 - 2.27s - loss: 0.3747588 - acc: 0.82765967 - batches: 348
Epoch 6/15 - 2.25s - loss: 0.36140448 - acc: 0.83639526 - batches: 348
Epoch 7/15 - 2.27s - loss: 0.34561557 - acc: 0.84915346 - batches: 348
Epoch 8/15 - 2.27s - loss: 0.32962668 - acc: 0.8586095 - batches: 348
Epoch 9/15 - 2.28s - loss: 0.3143761 - acc: 0.8646434 - batches: 348
Epoch 10/15 - 2.27s - loss: 0.2980835 - acc: 0.8698667 - batches: 348
Epoch 11/15 - 2.27s - loss: 0.28039435 - acc: 0.87463975 - batches: 348
Epoch 12/15 - 2.26s - loss: 0.26532406 - acc: 

In [79]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [80]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category_array, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [81]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1003
['pos']     415
Name: result, dtype: int64


In [82]:
preds_df[preds_df.result.apply(len)==2]

Empty DataFrame
Columns: [category, text, result, metadata]
Index: []


In [83]:
preds_df[preds_df.result.apply(len)==0]

Empty DataFrame
Columns: [category, text, result, metadata]
Index: []


MultiClassifierDL is a multi-label classifier, so some predictions may include both labels or none of the labels. That can be controlled a bit with `.setThreshold()` parameter during training. For now we will keep not keep zero label predictions and get the highest score as prediction.

In [84]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    1003
pos     415
Name: result, dtype: int64


In [85]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.86      0.85      0.86      1019
         pos       0.63      0.66      0.65       399

    accuracy                           0.80      1418
   macro avg       0.75      0.75      0.75      1418
weighted avg       0.80      0.80      0.80      1418


### Generic Classifier

In [88]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

GenericClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericClassifierApproach takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [89]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(128)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])



In [90]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [93]:
import os

log_files = os.listdir("./training_logs")

print(log_files)
with open("./training_logs/"+log_files[1]) as log_file:   #choose indexnumber of log file starting with GenericClassifierApproach
    print(log_file.read())

['MultiClassifierDLApproach_8b9eaf62a8c6.log', 'GenericClassifierApproach_39f2a442d476.log', 'ClassifierDLApproach_60c5d3e0075e.log']
Training 25 epochs
Epoch 1/25	0.28s	Loss: 11.554304	ACC: 0.6579176
Epoch 2/25	0.09s	Loss: 10.309638	ACC: 0.71410567
Epoch 3/25	0.09s	Loss: 9.725678	ACC: 0.72582793
Epoch 4/25	0.09s	Loss: 9.397478	ACC: 0.73549956
Epoch 5/25	0.09s	Loss: 9.441718	ACC: 0.73354644
Epoch 6/25	0.09s	Loss: 9.126393	ACC: 0.74509805
Epoch 7/25	0.09s	Loss: 9.190901	ACC: 0.74757
Epoch 8/25	0.09s	Loss: 9.19724	ACC: 0.7465986
Epoch 9/25	0.09s	Loss: 9.148613	ACC: 0.7489172
Epoch 10/25	0.09s	Loss: 8.737309	ACC: 0.76623076
Epoch 11/25	0.09s	Loss: 9.03104	ACC: 0.7624882
Epoch 12/25	0.09s	Loss: 8.856274	ACC: 0.75467914
Epoch 13/25	0.09s	Loss: 8.754003	ACC: 0.77244526
Epoch 14/25	0.09s	Loss: 8.600636	ACC: 0.7751051
Epoch 15/25	0.09s	Loss: 8.307023	ACC: 0.785487
Epoch 16/25	0.10s	Loss: 8.292088	ACC: 0.78123605
Epoch 17/25	0.09s	Loss: 8.468455	ACC: 0.7734166
Epoch 18/25	0.09s	Loss: 8.068778	A

In [99]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [100]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [101]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print(classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.86      0.84      0.85      1019
         pos       0.61      0.65      0.63       399

    accuracy                           0.79      1418
   macro avg       0.74      0.74      0.74      1418
weighted avg       0.79      0.79      0.79      1418


In [102]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

In [103]:
spark.createDataFrame(results_df).show()

+------------------+------------------+
|      pos-f1-score|          accuracy|
+------------------+------------------+
|0.6931506849315068| 0.842031029619182|
|0.6461916461916463|0.7968970380818053|
|0.6292682926829268|0.7856135401974612|
+------------------+------------------+


### GenericLogRegClassifier

In [104]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [105]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [106]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [108]:
import os

log_files = os.listdir("./training_logs")
print(log_files)
with open("./training_logs/"+log_files[3]) as log_file:   #chooese the index number of .log file starting with GenericLogRegClassifierApproach
    print(log_file.read())

['MultiClassifierDLApproach_8b9eaf62a8c6.log', 'GenericClassifierApproach_39f2a442d476.log', 'ClassifierDLApproach_60c5d3e0075e.log', 'GenericLogRegClassifierApproach_f66a6978f006.log']
Training 20 epochs
Epoch 1/20	0.12s	Loss: 26.192495	ACC: 0.7044375
Epoch 2/20	0.04s	Loss: 24.509409	ACC: 0.71197504
Epoch 3/20	0.04s	Loss: 23.64061	ACC: 0.7218276
Epoch 4/20	0.04s	Loss: 23.049358	ACC: 0.7347092
Epoch 5/20	0.04s	Loss: 22.71779	ACC: 0.74092025
Epoch 6/20	0.04s	Loss: 22.50545	ACC: 0.74456537
Epoch 7/20	0.04s	Loss: 22.272604	ACC: 0.7421631
Epoch 8/20	0.04s	Loss: 22.272514	ACC: 0.74925846
Epoch 9/20	0.03s	Loss: 22.097445	ACC: 0.7484612
Epoch 10/20	0.03s	Loss: 22.024399	ACC: 0.7439318
Epoch 11/20	0.03s	Loss: 21.817656	ACC: 0.75175816
Epoch 12/20	0.03s	Loss: 21.837666	ACC: 0.752726
Epoch 13/20	0.03s	Loss: 21.813433	ACC: 0.7533596
Epoch 14/20	0.03s	Loss: 21.746014	ACC: 0.75281656
Epoch 15/20	0.03s	Loss: 21.65612	ACC: 0.7516676
Epoch 16/20	0.03s	Loss: 21.846258	ACC: 0.7506824
Epoch 17/20	0.03s	L

In [109]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [110]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [111]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.79      0.93      0.85      1019
         pos       0.67      0.36      0.46       399

    accuracy                           0.77      1418
   macro avg       0.73      0.64      0.66      1418
weighted avg       0.75      0.77      0.74      1418


In [112]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [113]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [114]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(21)\
    .setBatchSize(128)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [115]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [117]:
import os

log_files = os.listdir("./training_logs")
print(log_files)

with open("./training_logs/"+log_files[-1]) as log_file:  #choose the index of the *.log file  starting with GenericSVMClassifierApproach
    print(log_file.read())

['MultiClassifierDLApproach_8b9eaf62a8c6.log', 'GenericClassifierApproach_39f2a442d476.log', 'ClassifierDLApproach_60c5d3e0075e.log', 'GenericLogRegClassifierApproach_f66a6978f006.log', 'GenericSVMClassifierApproach_e402e9876e14.log']
Training 21 epochs
Epoch 1/21	0.12s	Loss: 27.887125	ACC: 0.6994555
Epoch 2/21	0.03s	Loss: 25.189148	ACC: 0.7274225
Epoch 3/21	0.03s	Loss: 24.80061	ACC: 0.7352419
Epoch 4/21	0.03s	Loss: 24.754045	ACC: 0.7407427
Epoch 5/21	0.03s	Loss: 24.176756	ACC: 0.75157714
Epoch 6/21	0.03s	Loss: 24.288534	ACC: 0.7490008
Epoch 7/21	0.03s	Loss: 24.172213	ACC: 0.75210977
Epoch 8/21	0.03s	Loss: 24.17517	ACC: 0.7557549
Epoch 9/21	0.03s	Loss: 24.220263	ACC: 0.7616213
Epoch 10/21	0.03s	Loss: 23.864067	ACC: 0.7605524
Epoch 11/21	0.03s	Loss: 23.949997	ACC: 0.7597378
Epoch 12/21	0.03s	Loss: 23.914875	ACC: 0.75823027
Epoch 13/21	0.03s	Loss: 23.981165	ACC: 0.7555635
Epoch 14/21	0.03s	Loss: 24.415968	ACC: 0.7513021
Epoch 15/21	0.03s	Loss: 24.181353	ACC: 0.76099455
Epoch 16/21	0.03s	

In [118]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [119]:
preds.printSchema()
preds.select(preds.prediction).show(5, truncate=False)
preds.select(preds.category, preds.prediction.result).show(5, truncate=False)

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- category_array: array (nullable = false)
 |    |-- element: string (containsNull = true)
 |-- prediction: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable

In [120]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.77      0.96      0.85      1019
         pos       0.73      0.26      0.39       399

    accuracy                           0.77      1418
   macro avg       0.75      0.61      0.62      1418
weighted avg       0.76      0.77      0.72      1418


In [121]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["pos"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [123]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")\
        .setEnableInMemoryStorage(True)

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [124]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [125]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
| Acute pleuropericarditis relevant to post-cardiac injury...|     neg|[{sentence_embeddings, 0, 160,  Acute pleuropericarditis ...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [126]:
log_folder="ADE_logs_healthcare_200d"

### ClassifierDL

In [127]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(16)\
        .setMaxEpochs(30)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(
    stages = [
        classifier_dl
    ])

In [128]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [129]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.88      0.94      0.91      1019
         pos       0.81      0.67      0.73       399

    accuracy                           0.86      1418
   macro avg       0.84      0.80      0.82      1418
weighted avg       0.86      0.86      0.86      1418


In [130]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [131]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [132]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [133]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [134]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [135]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']    1050
['pos']     368
Name: result, dtype: int64


In [136]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df["result"].value_counts()

neg    1050
pos     368
Name: result, dtype: int64


In [137]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.89      0.91      0.90      1019
         pos       0.76      0.70      0.73       399

    accuracy                           0.86      1418
   macro avg       0.83      0.81      0.82      1418
weighted avg       0.85      0.86      0.85      1418


In [138]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [139]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [140]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(40)\
    .setBatchSize(16)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [141]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [142]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.95      0.75      0.84      1019
         pos       0.58      0.90      0.71       399

    accuracy                           0.79      1418
   macro avg       0.77      0.83      0.77      1418
weighted avg       0.85      0.79      0.80      1418


In [143]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [144]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [145]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [146]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [147]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [148]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.83      0.93      0.88      1019
         pos       0.75      0.51      0.60       399

    accuracy                           0.81      1418
   macro avg       0.79      0.72      0.74      1418
weighted avg       0.81      0.81      0.80      1418


In [149]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [150]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [151]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [152]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [153]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [154]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.83      0.92      0.87      1019
         pos       0.72      0.51      0.59       399

    accuracy                           0.81      1418
   macro avg       0.77      0.71      0.73      1418
weighted avg       0.80      0.81      0.79      1418


In [155]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["pos"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

Now we will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [156]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [157]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| (4) The acute rehabilitation of a 21-year-old woman with...|     neg|[{sentence_embeddings, 0, 182,  (4) The acute rehabilitat...|
| A 16-year-old girl with erosive, polyarticular JRA showe...|     neg|[{sentence_embeddings, 0, 129,  A 16-year-old girl with e...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [158]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+--------+------------------------------------------------------------+
|                                                        text|category|                                         sentence_embeddings|
+------------------------------------------------------------+--------+------------------------------------------------------------+
| A 51-year-old-woman presented with chronic eosinophilia,...|     neg|[{sentence_embeddings, 0, 139,  A 51-year-old-woman prese...|
| Acute pleuropericarditis relevant to post-cardiac injury...|     neg|[{sentence_embeddings, 0, 160,  Acute pleuropericarditis ...|
+------------------------------------------------------------+--------+------------------------------------------------------------+
only showing top 2 rows


In [159]:
log_folder="ADE_logs_bert"

### ClassifierDL

In [160]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(32)\
        .setMaxEpochs(20)\
        .setLr(0.001)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [161]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [164]:
import os

log_files = os.listdir("./ADE_logs_bert")
print(log_files)

with open("./ADE_logs_bert/"+log_files[-1]) as log_file:  #choose the index of the *.log file  starting with GenericSVMClassifierApproach
    print(log_file.read())

['ClassifierDLApproach_1f28ca04dfe3.log']
Training started - epochs: 20 - learning_rate: 0.001 - batch_size: 32 - training_examples: 5555 - classes: 2
Epoch 0/20 - 1.09s - loss: 103.78765 - acc: 0.7070562 - batches: 174
Epoch 1/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 2/20 - 0.57s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 3/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 4/20 - 0.57s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 5/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 6/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 7/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 8/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 9/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 10/20 - 0.58s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 11/20 - 0.57s - loss: 103.50767 - acc: 0.7074175 - batches: 174
Epoch 12/20 - 0.57s

In [162]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.72      1.00      0.84      1019
         pos       0.00      0.00      0.00       399

    accuracy                           0.72      1418
   macro avg       0.36      0.50      0.42      1418
weighted avg       0.52      0.72      0.60      1418


In [163]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [165]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [166]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(32)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [167]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [168]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [169]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['neg']           1046
['pos']            371
['neg', 'pos']       1
Name: result, dtype: int64


In [170]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

neg    1047
pos     371
Name: result, dtype: int64


In [171]:
print (classification_report(preds_df["category"], preds_df["result"]))

              precision    recall  f1-score   support

         neg       0.91      0.93      0.92      1019
         pos       0.81      0.75      0.78       399

    accuracy                           0.88      1418
   macro avg       0.86      0.84      0.85      1418
weighted avg       0.88      0.88      0.88      1418


In [172]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [173]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [174]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.5)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [175]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [176]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.95      0.89      0.92      1019
         pos       0.76      0.87      0.81       399

    accuracy                           0.89      1418
   macro avg       0.86      0.88      0.87      1418
weighted avg       0.90      0.89      0.89      1418


In [177]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [178]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [179]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.01)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [181]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [182]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [183]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.94      0.92      1019
         pos       0.82      0.74      0.78       399

    accuracy                           0.88      1418
   macro avg       0.86      0.84      0.85      1418
weighted avg       0.88      0.88      0.88      1418


In [184]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [187]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [188]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(20)\
    .setBatchSize(128)\
    .setLearningRate(0.005)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [193]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 2, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [194]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [195]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      1019
         pos       0.78      0.73      0.76       399

    accuracy                           0.87      1418
   macro avg       0.84      0.83      0.83      1418
weighted avg       0.87      0.87      0.87      1418


In [196]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_bert"] = [res["pos"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [197]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("stem")

logreg = DocumentLogRegClassifierApproach()\
    .setInputCols("stem")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    stemmer,
    logreg])
doclogreg_model = clf_Pipeline.fit(trainingData)

In [198]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.87      0.87      1019
         pos       0.67      0.69      0.68       399

    accuracy                           0.82      1418
   macro avg       0.77      0.78      0.78      1418
weighted avg       0.82      0.82      0.82      1418


In [199]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["pos"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in ADE Dataset

Here is the all results of classifier models. Only positive ADE f1 scores and accuracy scores are presented.

In [200]:
results_df

                          pos-f1-score  accuracy
ClassifierDL_100d             0.693151  0.842031
MultiClassifierDL_100d        0.646192  0.796897
GenericClassifier_100d        0.629268  0.785614
GenericLogReg_100d            0.464812  0.769394
GenericSVM_100d               0.386740  0.765162
ClassifierDL_200d             0.729767  0.861072
MultiClassifierDL_200d        0.732725  0.855430
GenericClassifier_200d        0.709931  0.791961
GenericLogReg_200d            0.604790  0.813822
GenericSVM_200d               0.594993  0.806065
ClassifierDL_bert             0.000000  0.718618
MultiClassifierDL_bert        0.779221  0.880113
GenericClassifier_bert        0.814988  0.888575
GenericLogReg_bert            0.781579  0.882934
GenericSVM_bert               0.755844  0.867419
DocumentLogRegClassifier      0.679012  0.816643


# Mtsamples Dataset

## Load Dataset

In [30]:
#data source ===>  !curl --output ./mtsamples_classifier.csv https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/mtsamples_classifier.csv

#upload this file into the s3 bucket

In [203]:
spark_df = spark.read.csv(f"{bucket}/mtsamples_classifier.csv", header = True)

spark_df.show(10,truncate=100)

+----------------+----------------------------------------------------------------------------------------------------+
|        category|                                                                                                text|
+----------------+----------------------------------------------------------------------------------------------------+
|Gastroenterology| PROCEDURES PERFORMED: Colonoscopy. INDICATIONS: Renewed symptoms likely consistent with active f...|
|Gastroenterology| OPERATION 1. Ivor-Lewis esophagogastrectomy. 2. Feeding jejunostomy. 3. Placement of two right-s...|
|Gastroenterology| PREOPERATIVE DIAGNOSES: 1. Gastroesophageal reflux disease. 2. Chronic dyspepsia. POSTOPERATIVE ...|
|Gastroenterology| PROCEDURE: Colonoscopy. PREOPERATIVE DIAGNOSES: Rectal bleeding and perirectal abscess. POSTOPER...|
|Gastroenterology| PREOPERATIVE DIAGNOSIS: Right colon tumor. POSTOPERATIVE DIAGNOSES: 1. Right colon cancer. 2. As...|
|Gastroenterology| PREOPERATIVE DIAGNOSI

In [204]:
spark_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- text: string (nullable = true)


In [205]:
spark_df.count()

638


In [206]:
spark_df.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|  115|
|Gastroenterology|  157|
|      Orthopedic|  223|
|       Neurology|  143|
+----------------+-----+


In [207]:
(trainingData, testData) = spark_df.randomSplit([0.8, 0.2], seed = 42)

In [208]:
trainingData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   95|
|Gastroenterology|  132|
|      Orthopedic|  188|
|       Neurology|  121|
+----------------+-----+


In [209]:
testData.groupBy("category").count().show()

+----------------+-----+
|        category|count|
+----------------+-----+
|         Urology|   20|
|Gastroenterology|   25|
|      Orthopedic|   35|
|       Neurology|   22|
+----------------+-----+


## 100 Dimension Healthcare Embeddings (embeddings_healthcare_100d)



We will extract [healthcare_100d embeddings](https://nlp.johnsnowlabs.com/2020/05/29/embeddings_healthcare_100d_en.html) and use it in the classificaiton model training.

In [210]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_healthcare_100d","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")\
        .setEnableInMemoryStorage(True)

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,

    ])

embeddings_healthcare_100d download started this may take some time.
Approximate size to download 475.8 MB
[OK!]


In [211]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)
trainingData_with_embeddings = trainingData_with_embeddings.select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [212]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)
testData_with_embeddings = testData_with_embeddings.select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [213]:
testData_with_embeddings.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sentence_embeddings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)


In [214]:
testData_with_embeddings.select(testData_with_embeddings.sentence_embeddings.embeddings).show(3,truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                          sentence_embeddings.embeddings|
+------------------------------------------------------------------------------------------------------------------------+
|[[-0.010433162, 0.0127568655, 0.110687375, 0.1609855, -0.14818177, -0.050856464, 0.064748034, -0.019294674, -0.002178...|
|[[-0.0055608996, 0.018396137, 0.12305377, 0.13861515, -0.18651922, -0.07559964, 0.12978752, -0.0046452526, -0.0031408...|
|[[-0.019461514, 0.060639214, 0.11142021, 0.12826534, -0.16216077, -0.10416892, 0.17136306, -0.020363769, 0.01169337, ...|
+------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows


In [215]:
log_folder="Mt_logs_healthcare_100d"

### ClassifierDL

In [216]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("prediction")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(50)\
        .setLr(0.005)\
        .setDropout(0.3)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)\
        # .setValidationSplit(0.1)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [217]:
clfDL_model_hc100 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [218]:
preds = clfDL_model_hc100.transform(testData_with_embeddings)

In [219]:
preds_df = preds.select("category","text","prediction.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.82      0.72      0.77        25
       Neurology       0.73      0.73      0.73        22
      Orthopedic       0.87      0.94      0.90        35
         Urology       0.65      0.65      0.65        20

        accuracy                           0.78       102
       macro avg       0.77      0.76      0.76       102
    weighted avg       0.78      0.78      0.78       102


In [220]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df = pd.DataFrame(columns=["macro-f1-score","weighted-f1-score","accuracy"])
results_df.loc["ClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]
results_df

                   macro-f1-score  weighted-f1-score  accuracy
ClassifierDL_100d        0.761835           0.782282  0.784314


### MultiClassifierDL

In [221]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [222]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(40)\
  .setLr(5e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [223]:
multiClassifier_model_hc100 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [224]:
preds = multiClassifier_model_hc100.transform(testData_with_embeddings)

In [225]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']          31
['Neurology']           22
['Gastroenterology']    21
['Urology']             20
[]                       8
Name: result, dtype: int64


In [226]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          31
Neurology           22
Gastroenterology    21
Urology             20
Name: result, dtype: int64


In [227]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.95      0.91      0.93        22
       Neurology       0.82      0.90      0.86        20
      Orthopedic       0.97      0.91      0.94        33
         Urology       0.90      0.95      0.92        19

        accuracy                           0.91        94
       macro avg       0.91      0.92      0.91        94
    weighted avg       0.92      0.91      0.92        94


In [228]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [229]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_100d.pb")

In [230]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_100d.pb")\
    .setEpochsNumber(50)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(False)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [231]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_100d.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_100d.pb


In [232]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [233]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.92      0.90        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.89      0.91      0.90        35
         Urology       0.94      0.85      0.89        20

        accuracy                           0.88       102
       macro avg       0.88      0.88      0.88       102
    weighted avg       0.88      0.88      0.88       102


In [234]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [235]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [236]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [237]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [238]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [239]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.91      0.84      0.87        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.86      0.91      0.89        35
         Urology       0.85      0.85      0.85        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.86      0.86      0.86       102


In [240]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [241]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [242]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [243]:
generic_model_hc100 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 100, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [244]:
pred_df = generic_model_hc100.transform(testData_with_embeddings)

In [245]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.88      0.88        25
       Neurology       0.89      0.77      0.83        22
      Orthopedic       0.82      0.94      0.88        35
         Urology       0.94      0.85      0.89        20

        accuracy                           0.87       102
       macro avg       0.89      0.86      0.87       102
    weighted avg       0.88      0.87      0.87       102


In [246]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_100d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## 200 Dimension Healthcare Embeddings (embeddings_clinical)



Now we will extract [embeddings_clinical](https://nlp.johnsnowlabs.com/2020/01/28/embeddings_clinical_en.html) embeddings which has 200 dimension output and use this embeddings in the model training.

In [247]:
document_assembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

tokenizer = Tokenizer() \
            .setInputCols(["document"]) \
            .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["document","token"])\
        .setOutputCol("word_embeddings")

sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "word_embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("AVERAGE")

embeddings_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        word_embeddings,
        sentence_embeddings,
    ])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


In [248]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")
trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [249]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")
testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [250]:
log_folder="Mt_logs_healthcare_200d"

### ClassifierDL

In [251]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(32)\
        .setMaxEpochs(30)\
        .setLr(0.01)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [252]:
clfDL_model_hc200 = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [253]:
preds = clfDL_model_hc200.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.90      0.72      0.80        25
       Neurology       0.72      0.59      0.65        22
      Orthopedic       0.83      0.97      0.89        35
         Urology       0.65      0.75      0.70        20

        accuracy                           0.78       102
       macro avg       0.78      0.76      0.76       102
    weighted avg       0.79      0.78      0.78       102


In [254]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL



In [255]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [256]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(16)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.2)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [257]:
multiClassifier_model_hc200 = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [258]:
preds = multiClassifier_model_hc200.transform(testData_with_embeddings)

In [259]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']                       22
['Gastroenterology']                 21
['Neurology']                        20
['Urology']                          17
['Orthopedic', 'Neurology']          15
['Gastroenterology', 'Neurology']     3
['Urology', 'Gastroenterology']       3
[]                                    1
Name: result, dtype: int64


In [260]:
# We will get the highest score label as result. you can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Neurology           29
Orthopedic          29
Gastroenterology    24
Urology             19
Name: result, dtype: int64


In [261]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.66      0.90      0.76        21
      Orthopedic       0.93      0.77      0.84        35
         Urology       0.89      0.85      0.87        20

        accuracy                           0.84       101
       macro avg       0.85      0.85      0.84       101
    weighted avg       0.86      0.84      0.85       101


In [262]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [263]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_200d.pb")

In [264]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_200d.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(32)\
    .setLearningRate(0.001)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.3)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [265]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_200d.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_200d.pb


In [266]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.88      0.92        25
       Neurology       0.89      0.73      0.80        22
      Orthopedic       0.85      0.94      0.89        35
         Urology       0.86      0.95      0.90        20

        accuracy                           0.88       102
       macro avg       0.89      0.88      0.88       102
    weighted avg       0.89      0.88      0.88       102


In [267]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [268]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [269]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [270]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [271]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [272]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.84      0.86        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.86      0.91      0.89        35
         Urology       0.84      0.80      0.82        20

        accuracy                           0.85       102
       macro avg       0.85      0.84      0.85       102
    weighted avg       0.85      0.85      0.85       102


In [273]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericLogReg_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [274]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [275]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(45)\
    .setBatchSize(32)\
    .setLearningRate(0.02)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [276]:
generic_model_hc200 = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 200, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [277]:
pred_df = generic_model_hc200.transform(testData_with_embeddings)

In [278]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.88      0.84      0.86        25
       Neurology       0.86      0.82      0.84        22
      Orthopedic       0.91      0.91      0.91        35
         Urology       0.82      0.90      0.86        20

        accuracy                           0.87       102
       macro avg       0.87      0.87      0.87       102
    weighted avg       0.87      0.87      0.87       102


In [279]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericSVM_200d"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Bert Sentence Embeddings (sbiobert_base_cased_mli)

We will extract [sbiobert_base_cased_mli](https://nlp.johnsnowlabs.com/2020/11/27/sbiobert_base_cased_mli_en.html) embeddings which has 768 dimension output and use this embeddings in the model training.

In [280]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

embeddings_pipeline = Pipeline(
    stages = [document_assembler,
              bert_sent])

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


In [281]:
trainingData_with_embeddings = embeddings_pipeline.fit(trainingData).transform(trainingData)\
                                                  .select("text","category","sentence_embeddings")

trainingData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMISSION DIAGNOSIS: Symptomatic cholelithiasis. DISCHAR...|Gastroenterology|[{sentence_embeddings, 0, 2228,  ADMISSION DIAGNOSIS: Sym...|
| ADMITTING DIAGNOSES: Hiatal hernia, gastroesophageal ref...|Gastroenterology|[{sentence_embeddings, 0, 3237,  ADMITTING DIAGNOSES: Hia...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [282]:
testData_with_embeddings = embeddings_pipeline.fit(testData).transform(testData)\
                                                  .select("text","category","sentence_embeddings")

testData_with_embeddings.show(2,truncate=60)

+------------------------------------------------------------+----------------+------------------------------------------------------------+
|                                                        text|        category|                                         sentence_embeddings|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
| ADMITTING DIAGNOSIS: Gastrointestinal bleed. HISTORY OF ...|Gastroenterology|[{sentence_embeddings, 0, 3978,  ADMITTING DIAGNOSIS: Gas...|
| CHIEF COMPLAINT: Dysphagia and hematemesis while vomitin...|Gastroenterology|[{sentence_embeddings, 0, 6515,  CHIEF COMPLAINT: Dysphag...|
+------------------------------------------------------------+----------------+------------------------------------------------------------+
only showing top 2 rows


In [288]:
log_folder="Mt_logs_bert"

### ClassifierDL

In [284]:
classifier_dl = ClassifierDLApproach()\
        .setInputCols(["sentence_embeddings"])\
        .setOutputCol("class")\
        .setLabelColumn("category")\
        .setBatchSize(8)\
        .setMaxEpochs(30)\
        .setLr(0.002)\
        .setDropout(0.1)\
        .setEnableOutputLogs(True)\
        .setOutputLogsPath(log_folder)

classifier_dl_pipeline = Pipeline(stages=[classifier_dl])

In [285]:
clfDL_model_bert = classifier_dl_pipeline.fit(trainingData_with_embeddings)

In [289]:
import os

log_files = os.listdir(f"./{log_folder}")
print(log_files)

with open(f"./{log_folder}/"+log_files[-1]) as log_file:  #choose the index of the *.log file  starting with ClassifierDLApproach
    print(log_file.read())

['ClassifierDLApproach_bc834c81a80b.log']
Training started - epochs: 30 - learning_rate: 0.002 - batch_size: 8 - training_examples: 536 - classes: 4
Epoch 0/30 - 0.78s - loss: 93.51679 - acc: 0.34514925 - batches: 67
Epoch 1/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 2/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 3/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 4/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 5/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 6/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 7/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 8/30 - 0.19s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 9/30 - 0.19s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 10/30 - 0.19s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 11/30 - 0.18s - loss: 92.825745 - acc: 0.35074627 - batches: 67
Epoch 12/30 - 0.18s - 

In [286]:
preds = clfDL_model_bert.transform(testData_with_embeddings)

preds_df = preds.select("category","text","class.result").toPandas()
preds_df["result"] = preds_df["result"].apply(lambda x : x[0])

print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       0.00      0.00      0.00        25
       Neurology       0.00      0.00      0.00        22
      Orthopedic       0.34      1.00      0.51        35
         Urology       0.00      0.00      0.00        20

        accuracy                           0.34       102
       macro avg       0.09      0.25      0.13       102
    weighted avg       0.12      0.34      0.18       102


In [290]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["ClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### MultiClassifierDL

In [291]:
# MultiClassifierDL accepts list of strings as input label. So we convert label column to array type.
trainingData_with_embeddings = trainingData_with_embeddings.withColumn("category_array", F.array(F.col("category")))
testData_with_embeddings = testData_with_embeddings.withColumn("category_array", F.array(F.col("category")))

In [292]:
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("prediction")\
  .setLabelColumn("category_array")\
  .setBatchSize(8)\
  .setMaxEpochs(20)\
  .setLr(9e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setOutputLogsPath(log_folder)\
#   .setValidationSplit(0.1)

multiClassifier_pipeline = Pipeline(
    stages = [
        multiClassifier
    ])

In [297]:
multiClassifier_model_bert = multiClassifier_pipeline.fit(trainingData_with_embeddings)

In [294]:
preds = multiClassifier_model_bert.transform(testData_with_embeddings)

In [295]:
preds_df = preds.select("category","text","prediction.result","prediction.metadata").toPandas()
preds_df.result.apply(lambda x: str(x) ).value_counts()

['Orthopedic']          36
['Gastroenterology']    22
['Urology']             22
['Neurology']           18
[]                       4
Name: result, dtype: int64


In [296]:
# We will get the highest score label as result. You can control the number of zero label results with setThreshold() in the training.
preds_df["scores"] = preds_df.metadata.apply(lambda x: {k:float(v) for k,v in x[0].items()} if len(x)>=1 else "")
preds_df["result"] = preds_df.scores.apply(lambda x: max(x, key=x.get) if len(x)>=1 else "")
preds_df = preds_df[(preds_df["result"]!="")]
preds_df["result"].value_counts()

Orthopedic          36
Gastroenterology    22
Urology             22
Neurology           18
Name: result, dtype: int64


In [299]:
print (classification_report(preds_df["category"], preds_df["result"]))

                  precision    recall  f1-score   support

Gastroenterology       1.00      0.88      0.94        25
       Neurology       0.89      0.76      0.82        21
      Orthopedic       0.86      0.94      0.90        33
         Urology       0.82      0.95      0.88        19

        accuracy                           0.89        98
       macro avg       0.89      0.88      0.88        98
    weighted avg       0.89      0.89      0.89        98


In [300]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["MultiClassifierDL_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### Generic Classifier

In [301]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_graph_builder = TFGraphBuilder()\
    .setModelName("generic_classifier")\
    .setInputCols(["features"])\
    .setLabelColumn("category")\
    .setHiddenLayers([300,200, 50])\
    .setHiddenAct("tanh")\
    .setHiddenActL2(True)\
    .setBatchNorm(True)\
    .setGraphFolder(graph_folder)\
    .setGraphFile("gcf_graph_bert.pb")

In [302]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

gen_clf = GenericClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("features")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/gcf_graph_bert.pb")\
    .setEpochsNumber(25)\
    .setBatchSize(16)\
    .setLearningRate(0.002)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.2)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_graph_builder,
    gen_clf])


In [303]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: generic_classifier
Graph folder: gc_graph
Graph file name: gcf_graph_bert.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [300, 200, 50], 'hidden_act': 'tanh', 'hidden_act_l2': True, 'batch_norm': True}
generic_classifier graph exported to gc_graph/gcf_graph_bert.pb


In [304]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.82      0.82      0.82        22
      Orthopedic       0.77      0.86      0.81        35
         Urology       0.88      0.75      0.81        20

        accuracy                           0.84       102
       macro avg       0.86      0.84      0.84       102
    weighted avg       0.85      0.84      0.84       102


In [305]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericClassifier_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericLogRegClassifier

In [306]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_logreg_graph_builder = TFGraphBuilder()\
    .setModelName("logreg_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("log_reg_graph.pb")

GenericLogRegClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericLogRegClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [307]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericLogRegClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/log_reg_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_logreg_graph_builder,
    gen_clf])


In [308]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: logreg_classifier
Graph folder: gc_graph
Graph file name: log_reg_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid'}
logreg_classifier graph exported to gc_graph/log_reg_graph.pb


In [309]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [310]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.86      0.86      0.86        35
         Urology       0.83      0.95      0.88        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.86      0.86      0.86       102


In [311]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCLogReg_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

### GenericSVMClassifier

In [312]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder = "gc_graph"

gc_svm_graph_builder = TFGraphBuilder()\
    .setModelName("svm_classifier")\
    .setInputCols(["feature_vector"]) \
    .setLabelColumn("category")\
    .setGraphFolder(graph_folder)\
    .setGraphFile("svm_graph.pb")

GenericSVMClassifier needs outputs from FeaturesAssembler. The FeaturesAssembler is used to collect features from different columns or an embeddings column.

The GenericSVMClassifier takes FEATURE_VECTOR annotations as input, classifies them and outputs CATEGORY annotations.

In [313]:
features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("feature_vector")

gen_clf = GenericSVMClassifierApproach()\
    .setLabelColumn("category")\
    .setInputCols("feature_vector")\
    .setOutputCol("prediction")\
    .setModelFile(f"{graph_folder}/svm_graph.pb")\
    .setEpochsNumber(30)\
    .setBatchSize(64)\
    .setLearningRate(0.004)\
    .setOutputLogsPath(log_folder)\
    .setDropout(0.1)\
    .setFixImbalance(True)\
    # .setValidationSplit(0.1)

clf_Pipeline = Pipeline(stages=[
    features_asm,
    gc_svm_graph_builder,
    gen_clf])


In [314]:
generic_model_bert = clf_Pipeline.fit(trainingData_with_embeddings)

TF Graph Builder configuration:
Model name: svm_classifier
Graph folder: gc_graph
Graph file name: svm_graph.pb
Build params: {'input_dim': 768, 'output_dim': 4, 'hidden_layers': [], 'output_act': 'sigmoid', 'loss_func': 'hinge'}
svm_classifier graph exported to gc_graph/svm_graph.pb


In [315]:
pred_df = generic_model_bert.transform(testData_with_embeddings)

In [316]:
preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.96      0.92      0.94        25
       Neurology       0.83      0.86      0.84        22
      Orthopedic       0.88      0.86      0.87        35
         Urology       0.86      0.90      0.88        20

        accuracy                           0.88       102
       macro avg       0.88      0.89      0.88       102
    weighted avg       0.88      0.88      0.88       102


In [317]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["GenericCSVM_bert"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## DocumentLogRegClassifier

In [318]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

logreg = DocumentLogRegClassifierApproach()\
    .setInputCols("token")\
    .setLabelCol("category")\
    .setOutputCol("prediction")\
    .setMaxIter(10)\
    .setTol(1e-6)

clf_Pipeline = Pipeline(stages=[document_assembler,
                                tokenizer,
                                logreg])

doclogreg_model = clf_Pipeline.fit(trainingData)

In [319]:
pred_df = doclogreg_model.transform(testData)

preds_df = pred_df.select('category','prediction.result').toPandas()
preds_df['result'] = preds_df.result.apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

                  precision    recall  f1-score   support

Gastroenterology       0.92      0.88      0.90        25
       Neurology       0.80      0.73      0.76        22
      Orthopedic       0.91      0.89      0.90        35
         Urology       0.79      0.95      0.86        20

        accuracy                           0.86       102
       macro avg       0.86      0.86      0.86       102
    weighted avg       0.87      0.86      0.86       102


In [320]:
res = classification_report(preds_df["category"], preds_df["result"], output_dict=True)
results_df.loc["DocumentLogRegClassifier"] = [res["macro avg"]["f1-score"], res["weighted avg"]["f1-score"], res["accuracy"]]

## Comparision of All Models' Result in Mtsamples Dataset

Here is the all results of classifier models. Macro f1, weighted f1 and accuracy scores are presented.

In [323]:
results_df

                          macro-f1-score  weighted-f1-score  accuracy
ClassifierDL_100d               0.761835           0.782282  0.784314
MultiClassifierDL_100d          0.911988           0.915787  0.914894
GenericClassifier_100d          0.879072           0.882285  0.882353
GenericLogReg_100d              0.858018           0.862609  0.862745
GenericSVM_100d                 0.871001           0.871947  0.872549
ClassifierDL_200d               0.760603           0.780091  0.784314
MultiClassifierDL_200d          0.843376           0.845308  0.841584
GenericClassifier_200d          0.878330           0.880668  0.882353
GenericLogReg_200d              0.846182           0.852450  0.852941
GenericSVM_200d                 0.866445           0.872451  0.872549
ClassifierDL_bert               0.127737           0.175326  0.343137
MultiClassifierDL_bert          0.883321           0.887450  0.887755
GenericClassifier_bert          0.844645           0.843765  0.843137
GenericCLogReg_bert 

In [37]:
# results_df.to_html().replace('\n','')

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>macro-f1-score</th>      <th>weighted-f1-score</th>      <th>accuracy</th>    </tr>  </thead>  <tbody>    <tr>      <th>ClassifierDL_100d</th>      <td>0.761835</td>      <td>0.782282</td>      <td>0.784314</td>    </tr>    <tr>      <th>MultiClassifierDL_100d</th>      <td>0.911988</td>      <td>0.915787</td>      <td>0.914894</td>    </tr>    <tr>      <th>GenericClassifier_100d</th>      <td>0.879072</td>      <td>0.882285</td>      <td>0.882353</td>    </tr>    <tr>      <th>GenericLogReg_100d</th>      <td>0.858018</td>      <td>0.862609</td>      <td>0.862745</td>    </tr>    <tr>      <th>GenericSVM_100d</th>      <td>0.871001</td>      <td>0.871947</td>      <td>0.872549</td>    </tr>    <tr>      <th>ClassifierDL_200d</th>      <td>0.760603</td>      <td>0.780091</td>      <td>0.784314</td>    </tr>    <tr>      <th>MultiClassifierDL_200d</th>      <td>0.843376</td>      <td>0.845308</td>      <td>0.841584</td>    </tr>    <tr>      <th>GenericClassifier_200d</th>      <td>0.878330</td>      <td>0.880668</td>      <td>0.882353</td>    </tr>    <tr>      <th>GenericLogReg_200d</th>      <td>0.846182</td>      <td>0.852450</td>      <td>0.852941</td>    </tr>    <tr>      <th>GenericSVM_200d</th>      <td>0.866445</td>      <td>0.872451</td>      <td>0.872549</td>    </tr>    <tr>      <th>ClassifierDL_bert</th>      <td>0.127737</td>      <td>0.175326</td>      <td>0.343137</td>    </tr>    <tr>      <th>MultiClassifierDL_bert</th>      <td>0.883321</td>      <td>0.887450</td>      <td>0.887755</td>    </tr>    <tr>      <th>GenericClassifier_bert</th>      <td>0.844645</td>      <td>0.843765</td>      <td>0.843137</td>    </tr>    <tr>      <th>GenericCLogReg_bert</th>      <td>0.860386</td>      <td>0.861821</td>      <td>0.862745</td>    </tr>    <tr>      <th>GenericCSVM_bert</th>      <td>0.882708</td>      <td>0.882774</td>      <td>0.882353</td>    </tr>    <tr>      <th>DocumentLogRegClassifier</th>      <td>0.855513</td>      <td>0.862087</td>      <td>0.862745</td>    </tr>  </tbody></table>